In [1]:
# CELL 1: SETUP AND IMPORTS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Setting up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

print("WORLD HAPPINESS REPORT - DATA CLEANING")

# Loading the raw dataset
print("\nLoading raw dataset...")
df_raw = pd.read_csv('../data/raw/world_happiness.csv', sep=';', decimal=',')
print(f"Raw dataset loaded: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")

# Creating a copy for cleaning (preserving original dataset)
df = df_raw.copy()
print("Working copy created for cleaning")

WORLD HAPPINESS REPORT - DATA CLEANING

Loading raw dataset...
Raw dataset loaded: 1,502 rows × 11 columns
Working copy created for cleaning


In [2]:
# CELL 2: INITIAL DATA QUALITY ASSESSMENT
# Identifying data quality issues that need to be addressed
print('Missing values summary:')
missing_summary = df.isnull().sum()
missing_percent =( missing_summary / len(df)) * 100

missing_df = pd.DataFrame({
    'Column' : missing_summary.index,
    'Missing Count': missing_summary.values,
    'Missing Percentage': missing_percent.values
}).sort_values('Missing Count', ascending=False)

for _, row in missing_df.iterrows():
    if row['Missing Count'] > 0:
        print(f"{row['Column']}: {row['Missing Count']} missing values ({row['Missing Percentage']:.1f}%)")
    else:
        print(f"{row['Column']}: No missing values")

# Checking for duplicates
duplicates = df.duplicated(['Country', 'Year']).sum()
print(f"\nDuplicate country-year combinations: {duplicates}")

# Checking data types
print(f"\nData Types:")
for col, dtype in df.dtypes.items():
    print(f"  {col}: {dtype}")

# Checking for obvious data quality issues
print(f"\nPOTENTIAL DATA QUALITY ISSUES:")

# 1. Checking happiness score range
happiness_range = [df['Happiness score'].min(), df['Happiness score'].max()]
if happiness_range[0] < 0 or happiness_range[1] > 10:
    print(f"Happiness scores outside 0-10 range: {happiness_range[0]:.3f} to {happiness_range[1]:.3f}")
else:
    print(f"Happiness scores in valid range: {happiness_range[0]:.3f} to {happiness_range[1]:.3f}")

# 2. Checking for negative values in columns that shouldn't have them
negative_cols = ['GDP per capita', 'Social support', 'Healthy life expectancy', 
                'Freedom to make life choices', 'Generosity']

for col in negative_cols:
    if col in df.columns:
        negative_count = (df[col] < 0).sum()
        if negative_count > 0:
            print(f"{col}: {negative_count} negative values")
        else:
            print(f"{col}: No negative values")

# 3. Checking for unrealistic values
print(f"\nValue Range Analysis:")
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col not in ['Ranking', 'Year']:
        min_val, max_val = df[col].min(), df[col].max()
        print(f"  {col}: {min_val:.3f} to {max_val:.3f}")

Missing values summary:
Regional indicator: 3 missing values (0.2%)
Ranking: No missing values
Country: No missing values
Happiness score: No missing values
GDP per capita: No missing values
Social support: No missing values
Healthy life expectancy: No missing values
Freedom to make life choices: No missing values
Generosity: No missing values
Perceptions of corruption: No missing values
Year: No missing values

Duplicate country-year combinations: 0

Data Types:
  Ranking: int64
  Country: object
  Regional indicator: object
  Happiness score: float64
  GDP per capita: float64
  Social support: float64
  Healthy life expectancy: int64
  Freedom to make life choices: float64
  Generosity: float64
  Perceptions of corruption: float64
  Year: int64

POTENTIAL DATA QUALITY ISSUES:
Happiness scores in valid range: 1.721 to 7.842
GDP per capita: No negative values
Social support: No negative values
Healthy life expectancy: No negative values
Freedom to make life choices: No negative values


In [3]:
# CELL 3: HANDLING MISSING VALUES
# Cleaning or imputing missing data based on patterns found
# =============================================================================

print("\nHANDLING MISSING VALUES")

# Since our initial assessment showed no missing values, we'll verify this
if df.isnull().sum().sum() == 0:
    print("No missing values found - dataset is complete!")
    print("No missing value imputation needed")
else:
    print("Addressing missing values...")
    
    # Strategy for missing values (if any exist):
    # 1. For happiness scores: Cannot impute (too important) - would remove rows
    # 2. For other metrics: Could use median by region or forward fill
    
    # Example missing value handling (uncomment if needed):
    # # Remove rows with missing happiness scores
    # df = df.dropna(subset=['Happiness score'])
    # 
    # # Impute other missing values with regional medians
    # for col in ['GDP per capita', 'Social support', 'Healthy life expectancy']:
    #     if col in df.columns and df[col].isnull().sum() > 0:
    #         df[col] = df.groupby('Regional indicator')[col].transform(
    #             lambda x: x.fillna(x.median())
    #         )

print(f"\nAfter missing value handling:")
print(f"  Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"  Total missing values: {df.isnull().sum().sum()}")


HANDLING MISSING VALUES
Addressing missing values...

After missing value handling:
  Dataset shape: 1,502 rows × 11 columns
  Total missing values: 3


In [4]:
# CELL 4: STANDARDIZING COUNTRY NAMES
# Ensuring consistent country naming across years
print("\nSTANDARDIZING COUNTRY NAMES")

# Checking for potential country name inconsistencies
print("Analyzing country name consistency...")

# Looking for countries with varying counts across years (potential name changes)
country_year_counts = df.groupby('Country')['Year'].count().sort_values()
expected_years = df['Year'].nunique()

print(f"Expected years per country: {expected_years}")
print(f"Countries with fewer than {expected_years} years of data:")

inconsistent_countries = country_year_counts[country_year_counts < expected_years]
if len(inconsistent_countries) > 0:
    for country, count in inconsistent_countries.head(10).items():
        print(f"  {country}: {count} years")
    print(f"... and {len(inconsistent_countries) - 10} more countries" if len(inconsistent_countries) > 10 else "")
else:
    print("All countries have consistent representation")

# Checking for similar country names that might be duplicates
print(f"\nChecking for similar country names...")
countries = sorted(df['Country'].unique())

# Looking for potential naming issues
naming_issues = []
for i, country in enumerate(countries):
    # Checking for countries with very similar names
    similar = [c for c in countries[i+1:] if 
              len(set(country.lower().split()) & set(c.lower().split())) > 0]
    if similar:
        naming_issues.append((country, similar))

if naming_issues:
    print("Potential naming inconsistencies found:")
    for main_country, similar_countries in naming_issues[:5]:  # Show first 5
        print(f"  {main_country} similar to: {similar_countries}")
else:
    print("No obvious country name inconsistencies found")

# Manual country name standardization (if needed)
country_name_mapping = {
    # Add any mappings here if inconsistencies are found
    # 'Old Name': 'New Name',
    # 'United States of America': 'United States',
    # 'Republic of Korea': 'South Korea',
}

if country_name_mapping:
    print(f"\nApplying country name standardization...")
    df['Country'] = df['Country'].replace(country_name_mapping)
    print(f"{len(country_name_mapping)} country names standardized")
else:
    print("\nNo country name standardization needed")


STANDARDIZING COUNTRY NAMES
Analyzing country name consistency...
Expected years per country: 10
Countries with fewer than 10 years of data:
  Djibouti: 1 years
  Congo: 1 years
  Puerto Rico: 1 years
  Oman: 1 years
  State of Palestine: 1 years
  Trinidad & Tobago: 1 years
  Algeria: 2 years
  Maldives: 2 years
  Suriname: 2 years
  Somaliland region: 2 years
... and 44 more countries

Checking for similar country names...
Potential naming inconsistencies found:
  Bosnia and Herzegovina similar to: ['Trinidad and Tobago']
  Central African Republic similar to: ['Czech Republic', 'Dominican Republic']
  China similar to: ['Hong Kong S.A.R. of China', 'Taiwan Province of China']
  Congo similar to: ['Congo (Brazzaville)', 'Congo (Kinshasa)']
  Congo (Brazzaville) similar to: ['Congo (Kinshasa)']

No country name standardization needed


In [5]:
# CELL 5: HANDLING OUTLIERS
# Identifying and address extreme values that might be data errors

print("\nOUTLIER DETECTION AND HANDLING")

# Defining columns to check for outliers
outlier_cols = ['Happiness score', 'GDP per capita', 'Social support', 
               'Healthy life expectancy', 'Freedom to make life choices', 
               'Generosity', 'Perceptions of corruption']

outliers_found = {}

print("Detecting outliers using IQR method...")

for col in outlier_cols:
    if col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outliers_found[col] = len(outliers)
        
        print(f"\n{col}:")
        print(f"  Valid range: {lower_bound:.3f} to {upper_bound:.3f}")
        print(f"  Outliers found: {len(outliers)}")
        
        if len(outliers) > 0:
            print(f"  Outlier values: {sorted(outliers[col].unique())}")
            # Show which countries have outliers
            outlier_countries = outliers[['Country', 'Year', col]].head(5)
            print(f"  Sample outlier cases:")
            for _, row in outlier_countries.iterrows():
                print(f"    {row['Country']} ({row['Year']}): {row[col]:.3f}")

# Outlier handling strategy
print(f"\nOUTLIER HANDLING STRATEGY:")
print("For this analysis, we'll keep outliers because:")
print("  1. Happiness data reflects real-world extremes")
print("  2. Some countries genuinely have extreme values")
print("  3. Outliers provide valuable insights")
print("  4. We have sufficient data points")

print(f"\nOutlier Summary:")
total_outliers = sum(outliers_found.values())
print(f"  Total outliers across all columns: {total_outliers}")
print(f"  Percentage of dataset: {(total_outliers / (len(df) * len(outlier_cols))) * 100:.2f}%")


OUTLIER DETECTION AND HANDLING
Detecting outliers using IQR method...

Happiness score:
  Valid range: 2.066 to 8.806
  Outliers found: 2
  Outlier values: [np.float64(1.721), np.float64(1.859)]
  Sample outlier cases:
    Afghanistan (2023): 1.859
    Afghanistan (2024): 1.721

GDP per capita:
  Valid range: -1.132 to 13.556
  Outliers found: 0

Social support:
  Valid range: 0.119 to 1.307
  Outliers found: 24
  Outlier values: [np.float64(0.0), np.float64(0.0145), np.float64(0.04198), np.float64(0.04592), np.float64(0.04829), np.float64(0.04981), np.float64(0.05329), np.float64(0.05507), np.float64(0.0794), np.float64(0.07965), np.float64(0.08805), np.float64(0.09327), np.float64(0.0998), np.float64(0.10054), np.float64(0.11284)]
  Sample outlier cases:
    Central African Republic (2015): 0.000
    Togo (2015): 0.100
    Afghanistan (2016): 0.093
    Benin (2016): 0.088
    Togo (2016): 0.000

Healthy life expectancy:
  Valid range: 47.000 to 87.000
  Outliers found: 19
  Outlier 

In [6]:
# CELL 6: DATA TYPE OPTIMIZATION
# Optimizing data types for better performance and memory usage

print("\nDATA TYPE OPTIMIZATION")

print("Current memory usage:")
memory_before = df.memory_usage(deep=True).sum() / 1024**2
print(f"  Total memory: {memory_before:.2f} MB")

# Optimizing data types
print(f"\nOptimizing data types...")

# Converting Year to integer if it's not already
if df['Year'].dtype != 'int64':
    df['Year'] = df['Year'].astype('int64')

# Converting Ranking to integer if it's not already  
if 'Ranking' in df.columns and df['Ranking'].dtype != 'int64':
    df['Ranking'] = df['Ranking'].astype('int64')

# Optimizing float columns (float64 to float32 if precision allows)
float_cols = df.select_dtypes(include=['float64']).columns
for col in float_cols:
    # Check if values fit in float32 range
    if df[col].min() >= np.finfo(np.float32).min and df[col].max() <= np.finfo(np.float32).max:
        df[col] = df[col].astype('float32')

# Convert categorical columns to category type for memory efficiency
categorical_cols = ['Country', 'Regional indicator']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

print("Data type optimization complete")

print(f"\nAfter optimization:")
memory_after = df.memory_usage(deep=True).sum() / 1024**2
print(f"  Total memory: {memory_after:.2f} MB")
print(f"  Memory reduction: {((memory_before - memory_after) / memory_before) * 100:.1f}%")

print(f"\nOptimized data types:")
for col, dtype in df.dtypes.items():
    print(f"  {col}: {dtype}")


DATA TYPE OPTIMIZATION
Current memory usage:
  Total memory: 0.29 MB

Optimizing data types...
Data type optimization complete

After optimization:
  Total memory: 0.09 MB
  Memory reduction: 69.2%

Optimized data types:
  Ranking: int64
  Country: category
  Regional indicator: category
  Happiness score: float32
  GDP per capita: float32
  Social support: float32
  Healthy life expectancy: int64
  Freedom to make life choices: float32
  Generosity: float32
  Perceptions of corruption: float32
  Year: int64


In [7]:
# CELL 7: CREATING DERIVED FEATURES
# Generating additional useful columns for analysis

print("\nCREATING DERIVED FEATURES")

print("Creating new analytical features...")

# 1. Happiness categories
def categorize_happiness(score):
    if score >= 7.0:
        return 'Very Happy'
    elif score >= 6.0:
        return 'Happy'
    elif score >= 5.0:
        return 'Moderate'
    elif score >= 4.0:
        return 'Unhappy'
    else:
        return 'Very Unhappy'

df['Happiness Category'] = df['Happiness score'].apply(categorize_happiness)
print("Created 'Happiness Category' feature")

# 2. Decade indicator
df['Decade'] = (df['Year'] // 10) * 10
print("Created 'Decade' feature")

# 3. Year period (early, mid, late)
def get_period(year):
    if year <= 2017:
        return 'Early (2015-2017)'
    elif year <= 2020:
        return 'Mid (2018-2020)'
    else:
        return 'Late (2021-2024)'

df['Period'] = df['Year'].apply(get_period)
print("Created 'Period' feature")

# 4. GDP category
df['GDP Category'] = pd.cut(df['GDP per capita'], 
                           bins=4, 
                           labels=['Low GDP', 'Lower-Mid GDP', 'Upper-Mid GDP', 'High GDP'])
print("Created 'GDP Category' feature")

# 5. Regional happiness rank
df['Regional Happiness Rank'] = df.groupby(['Regional indicator', 'Year'])['Happiness score'].rank(ascending=False)
print("Created 'Regional Happiness Rank' feature")

# 6. Year-over-year happiness change
df = df.sort_values(['Country', 'Year'])
df['Happiness Change'] = df.groupby('Country')['Happiness score'].diff()
print("Created 'Happiness Change' feature")

print(f"\nNew features created:")
new_features = ['Happiness Category', 'Decade', 'Period', 'GDP Category', 
               'Regional Happiness Rank', 'Happiness Change']
for feature in new_features:
    if feature in df.columns:
        print(f"{feature}")

print(f"\nDataset now has {df.shape[1]} columns (was {df_raw.shape[1]})")


CREATING DERIVED FEATURES
Creating new analytical features...
Created 'Happiness Category' feature
Created 'Decade' feature
Created 'Period' feature
Created 'GDP Category' feature
Created 'Regional Happiness Rank' feature
Created 'Happiness Change' feature

New features created:
Happiness Category
Decade
Period
GDP Category
Regional Happiness Rank
Happiness Change

Dataset now has 17 columns (was 11)


In [8]:
# CELL 8: DATA VALIDATION
# Verifying data quality after cleaning

print("\nDATA VALIDATION AFTER CLEANING")

print("Final data quality checks...")

# 1. Checking for missing values
missing_after = df.isnull().sum().sum()
print(f"Missing values: {missing_after}")

# 2. Checking for duplicates
duplicates_after = df.duplicated(['Country', 'Year']).sum()
print(f"Duplicate country-year combinations: {duplicates_after}")

# 3. Checking data ranges
print(f"\nValue ranges after cleaning:")
print(f"Happiness score: {df['Happiness score'].min():.3f} to {df['Happiness score'].max():.3f}")
print(f"Years: {df['Year'].min()} to {df['Year'].max()}")
print(f"Countries: {df['Country'].nunique()}")
print(f"Regions: {df['Regional indicator'].nunique()}")

# 4. Checking data distribution
print(f"\nData distribution:")
print(f"Total observations: {len(df):,}")
print(f"Countries per year (avg): {len(df) / df['Year'].nunique():.1f}")
print(f"Years per country (avg): {len(df) / df['Country'].nunique():.1f}")

# 5. Validating new features
print(f"\nNew feature validation:")
print(f"Happiness categories: {df['Happiness Category'].value_counts().to_dict()}")
print(f"Periods: {df['Period'].value_counts().to_dict()}")
print(f"Non-null happiness changes: {df['Happiness Change'].notna().sum()}")


DATA VALIDATION AFTER CLEANING
Final data quality checks...
Missing values: 181
Duplicate country-year combinations: 0

Value ranges after cleaning:
Happiness score: 1.721 to 7.842
Years: 2015 to 2024
Countries: 175
Regions: 10

Data distribution:
Total observations: 1,502
Countries per year (avg): 150.2
Years per country (avg): 8.6

New feature validation:
Happiness categories: {'Moderate': 466, 'Unhappy': 368, 'Happy': 363, 'Very Unhappy': 161, 'Very Happy': 144}
Periods: {'Late (2021-2024)': 570, 'Early (2015-2017)': 470, 'Mid (2018-2020)': 462}
Non-null happiness changes: 1327


In [9]:
# CELL 9: SAVING CLEANED DATASET
# Exporting the cleaned data for use in subsequent analysis

print("\nSAVING CLEANED DATASET")

# Saving cleaned dataset
output_path = '../data/processed/happiness_cleaned.csv'
df.to_csv(output_path, index=False)
print(f"Cleaned dataset saved to: {output_path}")

# Saved data cleaning summary
summary_stats = {
    'original_rows': len(df_raw),
    'cleaned_rows': len(df),
    'original_columns': len(df_raw.columns),
    'cleaned_columns': len(df.columns),
    'countries': df['Country'].nunique(),
    'years': df['Year'].nunique(),
    'missing_values': df.isnull().sum().sum(),
    'duplicates': df.duplicated(['Country', 'Year']).sum(),
    'memory_mb': df.memory_usage(deep=True).sum() / 1024**2
}

print(f"\nCLEANING SUMMARY:")
print(f"Original dataset: {summary_stats['original_rows']:,} rows × {summary_stats['original_columns']} columns")
print(f"Cleaned dataset: {summary_stats['cleaned_rows']:,} rows × {summary_stats['cleaned_columns']} columns")
print(f"Data retention: {(summary_stats['cleaned_rows'] / summary_stats['original_rows']) * 100:.1f}%")
print(f"Countries: {summary_stats['countries']}")
print(f"Years: {summary_stats['years']}")
print(f"Missing values: {summary_stats['missing_values']}")
print(f"Duplicates: {summary_stats['duplicates']}")
print(f"Memory usage: {summary_stats['memory_mb']:.2f} MB")


SAVING CLEANED DATASET
Cleaned dataset saved to: ../data/processed/happiness_cleaned.csv

CLEANING SUMMARY:
Original dataset: 1,502 rows × 11 columns
Cleaned dataset: 1,502 rows × 17 columns
Data retention: 100.0%
Countries: 175
Years: 10
Missing values: 181
Duplicates: 0
Memory usage: 0.30 MB
